## Results Generation Notebook
This notebook can be run from start to finish without any user intervention. It loops through the datasets defined in `DATASETS` and fits each classifier listed in `CLASSIFIERS`. In the final cell eval metrics are generated for each classifier-dataset combination. 

Classifier definitions can be found in `classifiers.py`

Data loaders can be found in `data.py`

Evaluation metric definitions can be found in `eval.py`

In [1]:
from classifiers import ConformalPredictor, WCRF, RandomForest, NaiveCautiousClassifier
from data import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from eval import conformal_pred_eval, wcrf_eval, evaluate_model
import numpy as np


In [2]:
DATASETS = ["german_credit_data", "heart_disease_data", "breast_cancer_data", "compas_data"]
CLASSIFIERS = ['RandomForest', 'WCRF', 'ConformalPredictor', 'NaiveCautiousClassifier']

## Load pre-processed data

In [3]:
datasets = {}
for dataset_name in DATASETS:
    print(f"Processing dataset: {dataset_name}")
    data = load_dataset(dataset_name)
    datasets[dataset_name] = data

Processing dataset: german_credit_data
Processing dataset: heart_disease_data
Processing dataset: breast_cancer_data
Processing dataset: compas_data


## Initialize models

In [5]:
classifiers = {}
for dataset_name in DATASETS:
    X_train, X_calib, X_test, y_train, y_calib, y_test = datasets[dataset_name]["train"], datasets[dataset_name]["calib"], datasets[dataset_name]["test"], datasets[dataset_name]["train_labels"],datasets[dataset_name]["calib_labels"],datasets[dataset_name]["test_labels"]
    rfc = RandomForest(X_train, y_train)
    # Conformal RF
    cpc = ConformalPredictor()
    # Weighted Cautiouse Random Forest
    wcrf = WCRF(s=1,gamma=1,labda=5)
    # Naive Cautious Classifier
    ncc = NaiveCautiousClassifier(X_train, y_train, 0.7)
    classifiers[str(dataset_name)+'__RandomForest'] = rfc
    classifiers[str(dataset_name)+'__ConformalPredictor'] = cpc
    classifiers[str(dataset_name)+'__WCRF'] = wcrf
    classifiers[str(dataset_name)+'__NaiveCautiousClassifier'] = ncc



## Fit models

In [ ]:
## Get model params
## NOTE this took me ~1hr to run, so once you get params dont re-run this, just run the below!!
best_params_per_dataset = {} 
wcrf_params_per_dataset = {}
for classifier in classifiers.keys():
    # Vanilla Random Forest
    dataset_name = classifier.split('__')[0]
    if "RandomForest" in str(classifier): 
        if dataset_name not in best_params_per_dataset:
            best_params = classifiers[classifier].gridSearch()
            best_params_per_dataset[dataset_name] = best_params
        classifiers[classifier].fit(best_params_per_dataset[dataset_name])
    # Conformal RF
    elif "ConformalPredictor" in str(classifier):
        if dataset_name not in best_params_per_dataset:
            best_params = classifiers[classifier].gridSearch(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
            best_params_per_dataset[dataset_name] = best_params
        classifiers[classifier].fit(best_params_per_dataset[dataset_name], datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'], datasets[dataset_name]['calib'], datasets[dataset_name]['calib_labels'])
    # Weighted Cautiouse Random Forest
    elif "WCRF" in str(classifier):
        if dataset_name not in wcrf_params_per_dataset:
            classifiers[classifier].gridSearch(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'], datasets[dataset_name]['test'], datasets[dataset_name]['test_labels'])
            wcrf_params_per_dataset[dataset_name] = True
        classifiers[classifier].fit(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
        classifiers[classifier].fit_w(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
    elif "NaiveCautiousClassifier" in str(classifier):
        if dataset_name not in best_params_per_dataset:
            best_params = classifiers[classifier].gridSearch()
            best_params_per_dataset[dataset_name] = best_params
        classifiers[classifier].fit(best_params_per_dataset[dataset_name])
    else: 
        raise ValueError(f"Classifier: {classifier} not found")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_

In [ ]:
# Fit classifier with already obtained models params 
for classifier in classifiers.keys():
    # Vanilla Random Forest
    dataset_name = classifier.split('__')[0]
    if "RandomForest" in str(classifier): 
        classifiers[classifier].fit(best_params_per_dataset[dataset_name])
    # Conformal RF
    elif "ConformalPredictor" in str(classifier):
        classifiers[classifier].fit(best_params_per_dataset[dataset_name], datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'], datasets[dataset_name]['calib'], datasets[dataset_name]['calib_labels'])
    # Weighted Cautiouse Random Forest
    elif "WCRF" in str(classifier):
        classifiers[classifier].fit(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
        classifiers[classifier].fit_w(datasets[dataset_name]['train'], datasets[dataset_name]['train_labels'])
    elif "NaiveCautiousClassifier" in str(classifier):
        classifiers[classifier].fit(best_params_per_dataset[dataset_name])
    else: 
        raise ValueError(f"Classifier: {classifier} not found")

## Run Eval

In [11]:
for classifier in classifiers.keys():
    classifier_name = classifier.split("__")[1]
    dataset_name = classifier.split("__")[0]
    print("=" * 50)
    print(f'Evaluation Results for Classifier: {classifier_name}')
    print(f'Dataset: {dataset_name}')
    print('-' * 50)
    model_eval = evaluate_model(
        classifier_name, 
        datasets[dataset_name]['test'], 
        datasets[dataset_name]['test_labels'], 
        classifiers[classifier]
    )
    if isinstance(model_eval, dict):
        for metric, value in model_eval.items():
            print(f"{metric.replace('_', ' ').capitalize()}: {value}")
    else:
        print(f'Accuracy:  {model_eval}')
    print('='*50)


Evaluation Results for Classifier: RandomForest
Dataset: german_credit_data
--------------------------------------------------
Accuracy:  0.74
Evaluation Results for Classifier: ConformalPredictor
Dataset: german_credit_data
--------------------------------------------------
U65 score: 67.2
Single set accuracy: 83.33
Determinacy: 12.0
Abstention: -11.0
Evaluation Results for Classifier: WCRF
Dataset: german_credit_data
--------------------------------------------------
U65 score: 78.5
Single set accuracy: 78.5
Determinacy: 100.0
Precise accuracy: 78.5
Evaluation Results for Classifier: NaiveCautiousClassifier
Dataset: german_credit_data
--------------------------------------------------
U65 score: 77.25
Single set accuracy: 87.27
Determinacy: 55.0
Abstention: -54.0
Evaluation Results for Classifier: RandomForest
Dataset: heart_disease_data
--------------------------------------------------
Accuracy:  0.8618421052631579
Evaluation Results for Classifier: ConformalPredictor
Dataset: hear